# Deep Neural Networks

Artificial Neural Network (ANN) are a powerful tool, but what happens if we need to tackle **really complex problems**, such as detecting hundreds of types of objects in high-resolution images? We need to train a **deeper** network, perhaps with ten layers or many more, each containing hundreds of neurons, linked by hundreds of thousands of connections. The term **deep** in Deep Neural Networks (DNN) refers to the presence of those multiple hidden layers between the input and output, which enable the network to **learn hierarchical representations of data**, where higher layers capture **increasingly abstract features**. Other neural networks are often referred to as **shallow** in comparison to DNNs. Shallow neural networks typically have only one or two hidden layers, which limits their ability to learn complex patterns in data. While they can still be effective for simpler tasks, they may struggle with more intricate datasets where deeper architectures are needed to capture nuanced relationships. Training a DNN rise some problems:

- the gradients can grow smaller and smaller, or larger and larger, when flowing backward through the network during training, making lower layers very hard to train (**vanishing/explodin gradients**);
- we might not have enough training data for such a large network, or it might be too costly to label;
- training may be extremely slow;
- a model with millions of parameters would severely risk overfitting the training set.

In the following we will go through each of these problems and present techniques to solve them. 

## The Vanishing-Exploding Gradient Problem

The backpropagation algorithm works by going from the output layer to the input layer, propagating the error gradient along the way. Once the algorithm has computed the gradient of the cost function with regard to each parameter in the network, it uses these gradients to update each parameter with a gradient descent step. Unfortunately, **gradients often get smaller and smaller** as the algorithm progresses down to the lower layers. As a result, the update pass leaves the lower layers connection weights **virtually unchanged**, and training never converges to a good solution. We call this problem the **vanishing gradients problem**.  In some cases, the opposite can happen: the **gradients can grow bigger and bigger** until layers get insanely large weight update and the algorithm diverges. This is the **exploding gradients problem**. More generally, DNN suffer from **unstable gradients**:

![](images/vanishing-exploding-gradient.png)

The problem is due to the combination of the logistic sigmoid activation function $\sigma$ and the weight initialization technique ([Understanding the Difficulty of Training Deep Feedforward Neural Networks](http://proceedings.mlr.press/v9/glorot10a.html)). Backpropagation involves computing the partial derivatives of the loss function with respect to each weight in the network, representing their contribution to the overall loss:

$\displaystyle \frac{\partial L}{\partial w}$

using the chain rule, we can express the gradient of the loss function with respect to the weights of the output layer $l$ as:

$\displaystyle \frac{\partial L}{\partial w^{(l)}} = \frac{\partial L}{\partial \sigma(z^{(l)})} \cdot \frac{\partial \sigma(z^{(l)})}{\partial z^{(l)}} \cdot \frac{\partial z^{(l)}}{\partial w^{(l)}}$

then the gradient in the lower layer $(l-1)$ of the network is:

$\displaystyle \frac{\partial L}{\partial w^{(l-1)}} = \frac{\partial L}{\partial \sigma(z^{(l)})} \cdot \frac{\partial \sigma(z^{(l)})}{\partial z^{(l)}} \cdot \frac{\partial z^{(l)}}{\partial \sigma(z^{(l-1)})} \cdot \frac{\partial \sigma(z^{(l-1)})}{\partial z^{(l-1)}} \cdot \frac{\partial z^{(l-1)}}{\partial w^{(l-1)}}$

As we move backward through the layers, the chain rule causes **the gradient to be multiplied repeatedly**. This multiplication can lead to a significant increase or decrease in the gradient, depending on the values of the weights and the activation function. When we use the sigmoid activation function, which has a saturating nature, its derivative is much smaller than 1 for a wide range of input values. Consequently, the gradients of the weights in the lower layers of the network become the products of many small numbers.

In [1]:
import numpy as np

def logit(x):
    return 1 / (1 + np.exp(-x))

In [2]:
def derivative(f, x, h):
    return (f(x + h) - f(x)) / h

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(-5, 5, 200)

plt.plot(x, logit(x), "b-")
plt.plot(x, derivative(logit, x, 0.001), "g--")
plt.title("Logistic activation function and its derivative")
plt.show()

Due to the multiplication of these small values, the gradients associated with the weights in the lower layers gradually diminish in size. The small gradients make it challenging for the lower layers to receive meaningful updates during the training process, which can hinder the learning process and result in slower convergence or even stagnation in training progress.

The exploding gradient problem can occur when the product of the gradient values becomes larger than 1 as it propagates backward through the layers of the network. This can lead to gradients with magnitudes that grow exponentially, making training unstable and potentially causing numerical issues. When multiple layers have weights greater than 1, the gradients can grow exponentially as they propagate back through the network during training. 

I order to show the problem, we can try to train a fully connected network with 10 hidden layers over the Fashion MNIST dataset using the sigmoid activation function:

In [4]:
import keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [5]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(28, 28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(300, activation="sigmoid"))
for layer in range(10):
    model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [6]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

In [7]:
history = model.fit(X_train, y_train, epochs=10, verbose=0,
                    validation_data=(X_valid, y_valid))

In [ ]:
print("Validation accuracy:", history.history["val_accuracy"][-1])

As we can see, the accuracy of the model is very low, and the loss is not decreasing. This is due to the vanishing gradient problem.

### Initialization strategies

In order to alleviate the unstable gradients problem, we need **the signal to flow properly in both directions**: in the forward direction when making predictions, and in the reverse direction when backpropagating gradients. We don’t want the signal to die out, nor do we want it to explode and saturate. For the signal to flow properly, we need **the variance of the outputs of each layer to be equal to the variance of its inputs**, and we need **the gradients to have equal variance before and after flowing through a layer in the reverse direction**. Although it is impossible for both conditions to hold for any layer in the network until and unless the number of inputs to the layer (**fan-in**) is equal to the number of neurons in the layer (**fan-out**), researchers proposed a well-proven compromise that works incredibly well in practice: the connection weights of each layer must be initialized randomly as described in the following equations:

Normal distribution with mean $0$ and variance  $\sigma^2=\frac{1}{\text{fan}_\text{avg}}$  

Uniform distribution between $-r$ and $+r$ with $r=\sqrt{\frac{3}{\text{fan}_\text{avg}} }$ 

where $\text{fan}_\text{avg}=(\text{fan}_\text{in}+\text{fan}_\text{out})/2$

This initialization strategy is called **Xavier initialization** or **Glorot initialization**. By default, Keras uses Glorot initialization with a uniform distribution, however it support a number of other strategies. When creating a layer, we can change the initialization stratergy by setting **kernel_initializer** parameter:

In [ ]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

### Nonsaturating Activation Functions
The problem with unstable gradients is in part due to a poor choice of activation function. The sigmoid activation function (used in biological neurons) is not the best choice. The **ReLU activation function** does not saturate for positive values:

$\displaystyle \text{ReLU}(z) = \max(0,z)$

In [10]:
def relu(x):
    return np.maximum(0, x)

In [ ]:
plt.plot(x, relu(x), "b-", linewidth=2)
plt.title("ReLU activation function")
plt.show()

Unfortunately, it is not perfect. It suffers from a problem known as the **dying ReLUs**: during training, some neurons “die” (they stop outputting anything other than 0). This happens when the neuron weights get tweaked in a way that the weighted sum of inputs are negative for all instances in the training set. In these conditions, the neuron just keeps outputting zeros, and Gradient Descent does not affect it anymore because the gradient of the ReLU function is zero when its input is negative. To solve this problem, we can use a variant of the ReLU function: the **leaky ReLU**. 

$\displaystyle \text{LeakyReLU}(x)=\text{max}(\alpha x, x)$ 

The small slope for negative values ensures that neurons never die, weights can go into a "long coma", but they have a chance to eventually wake up. The hyperparameter defines how much the function “leaks”. 

In [12]:
def leaky_relu(x, alpha=0.01):
    return np.maximum(alpha*x, x)

In [ ]:
plt.plot(x, leaky_relu(x, 0.05), "b-", linewidth=2)
plt.title("Leaky ReLU activation function")
plt.show()

We can try to substitute the sigmoid activation function with the ReLU activation function in the previous fully connected network, and we can see that the model is now able to learn and the accuracy is increasing:

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(28, 28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(300, activation="relu"))
for layer in range(10):
    model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [15]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

In [16]:
history = model.fit(X_train, y_train, epochs=10, verbose=0,
                    validation_data=(X_valid, y_valid))

In [ ]:
print("Validation accuracy:", history.history["val_accuracy"][-1])

Other variants are: the **randomized leaky ReLU (RReLU)**, where the parameter is picked randomly in a given range during training and is fixed to an average value during testing, and the **parametric leaky ReLU (PReLU)**, where the parameter is learned during training. For a comparison of several variants of the ReLU activation function, we can refer to the paper:[**Empirical Evaluation of Rectified Activations in Convolutional Network**](https://arxiv.org/pdf/1505.00853.pdf). Another possibility is to use the **exponential linear unit (ELU)** activation function, that outperform ReLU variants since it provides a smoother and continuous gradient. The main drawback of the ELU activation function is that it is slower to compute than the ReLU functions (due to the use of the exponential function). Its faster convergence rate during training compensates for that slow computation, but still, at test time an ELU network will be slower than a ReLU network.

$\displaystyle \text{ELU}_\alpha(x) = \Bigg \{ \begin{matrix} \alpha(\text{exp}(x)-1 & \text{if} & x < 0 \\ x & \text{if} & x > 0 \end{matrix}$

In [18]:
def elu(x, alpha=1):
    return np.where(x < 0, alpha * (np.exp(x) - 1), x)

In [ ]:
plt.plot(x, elu(x), "b-", linewidth=2)
plt.title("ELU activation function")
plt.show()

Finally, the **Scaled ELU (SELU)** activation function:

$\displaystyle \text{SELU}_{\alpha, \lambda}(x) = \lambda \Bigg \{ \begin{matrix} \alpha(\text{exp}(x)-1 & \text{if} & x < 0 \\ x & \text{if} & x > 0 \end{matrix}$

where tipically 

$\displaystyle \alpha=1.67326$

$\displaystyle \lambda=1.0507$

The main advantage of SELU is its ability to create **self-normalizing neural networks**: the output of each layer will tend to preserve a mean of 0 and standard deviation of 1 during training.

In [20]:
def selu(x, lamba=1.00507, alpha=1.67326):
    return lamba * elu(x, alpha)

In [ ]:
plt.plot(x, selu(x), "b-", linewidth=2)
plt.title("SELU activation function")
plt.show()

So, which activation function should we use? In general 

**SELU -> ELU -> Leaky ReLU (and its variants) -> ReLU -> Tanh > Logistic**

However, sonce ReLU is the most used activation function, many libraries and hardware accelerators provide ReLU-specific optimizations; therefore, if speed is a priority, ReLU might  be the best choice. An update list of activation functions with refereces can be accessed [here](https://paperswithcode.com/methods/category/activation-functions). 

### Batch Normalization

Initialization techniques and ELU or variant of ReLU can significantly reduce the the vanishing/exploding gradients problems at the beginning of training however it doesn’t guarantee that they won’t come back during training. The [batch normalization technique](https://arxiv.org/abs/1502.03167) consists of adding an operation in the model before or after the activation function of each hidden layer. This operation **zero-centers, normalizes each input** using the mean and standard deviation of the input over the current mini-batch $B$ of size $m_B$ (hence the name “Batch Normalization”):

$\displaystyle \mu_{B}=\frac{1}{m_B}\sum\limits_{i=1}^{m_B}{x^{(i)}}$ 

$\displaystyle \sigma _B^2 =\frac{1}{m_B}\sum\limits_{i=1}^{m_B}{(x^{(i)}-\mu _B)^2}$ 

$\displaystyle \hat{x}^{(i)} =\frac{x^{(i)}-\mu _B}{\sqrt{\sigma _B^2+\epsilon}}$

However, these normalized values may not follow the original distribution. To tackle this, batch normalization introduces two learnable parameters, $\gamma$ and $\beta$, which can **shift and scale the normalized values**:

$\displaystyle z^{(i)}=\gamma \hat{x}^{(i)}+\beta$

So during training, batch normalization standardizes inputs, then rescales and offsets them. But what about at prediction time? We may need to make predictions for individual instances or over small batch of instances, so computing statistics would be unreliable.  One solution could be to wait until the end of training, then run the whole training set through the neural network and compute the mean and standard deviation of each input. These “final” means and standard deviations could then be used instead of the batch ones when making predictions. Most implementations estimate these statistics during training by using a moving average of the layer’s input means and standard deviations.

Batch normalization adds some complexity to the model and there is a runtime penalty, the network makes slower predictions due to the extra computations required at each layer. Fortunately, it’s  possible to fuse the batch normalization layer with the previous one, after training, in order to avoid the runtime penalty. This is done by updating the previous weights and biases to directly produces outputs of the appropriate scale and offset. The previous laryer computes: 

$\displaystyle XW+b$

the batch normalization layers computes: 

$\displaystyle \gamma \frac{(XW+b-\mu)}{\sigma} + \beta$

if we define:

$\displaystyle W' = \gamma \frac{W}{\sigma}$

$\displaystyle b'= \gamma \frac{(b-\mu)}{\sigma} +\beta$

the equation simplifies to

$\displaystyle XW'+b'$

so, if we replace the previous layer weights and biases with the updated weights and biases, we can get rid of the batch normalization layer.

In Keras, we can use batch normalization just adding a layer before or after each hidden layer activation function. As an example, the following model applies it after every hidden layer of the previous example using the sigmoid activation function, as you can see the model is able to learn: 

In [22]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(28, 28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(300, activation="sigmoid"))
for layer in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [23]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

In [24]:
history = model.fit(X_train, y_train, epochs=10, verbose=0,
                    validation_data=(X_valid, y_valid))

In [ ]:
print("Validation accuracy:", history.history["val_accuracy"][-1])

Batch normalization has become one of the most-used layers in DNN, to the point that it is often omitted in the diagrams, as it is assumed that it is added after every layer. However, a paper [Fixup Initialization: Residual Learning Without Normalization**](https://arxiv.org/abs/1901.09321) may change this assumption: by using a novel weight initialization technique, the authors managed to train a very deep network (10,000 layers) without batch normalization, achieving state-of-the-art performance on complex image classification tasks.

### Gradient Clipping

Another popular technique to mitigate the exploding gradients problem is to [**clip the gradients**](https://arxiv.org/abs/1211.5063) during backpropagation so that they never exceed some threshold. This technique is most often used in recurrent neural networks, for other types of networks, batch normalization is usually sufficient. In Keras, the use of Gradient Clipping is done by setting **clipvalue** or **clipnorm** arguments when creating an optimizer:

In [26]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

This optimizer will clip every component of the gradient vector to a value between –1.0 and 1.0. Note that this may change the orientation of the gradient vector. For instance, if the
original gradient vector is [0.9, 100.0], it points mostly in the direction of the second axis; but once you clip it by value, you get [0.9, 1.0], which points roughly in the diagonal between the two axes. In practice, this approach works well. If we want to ensure that Gradient Clipping does not change the direction of the gradient vector, we can clip by norm by setting clipnorm instead of clipvalue. This will clip the whole gradient if its norm is greater than the threshold.

## Transfer Learning

It is generally not a good idea to train a large DNN from scratch, since it requires a lot of data and computing power. A better approach is to find an existing neural network that accomplishes a similar task to the one we are trying to tackle, then reuse the lower layers of this network. This technique is called **transfer learning**. It will not only speedcup training considerably, but also require significantly less training data. Suppose we have access to a DNN that was trained to classify pictures into 100 different categories (e.g. animals, plants, vehicles, and everyday objects) and we want to train a DNN to classify specific types of vehicles. These tasks are very similar, even partly overlapping, so we can try to reuse parts of the first network:

![](images/transfer-learning.png)

The output layer of the original model should usually be replaced because it is most likely not useful at all for the new task, and it may not even have the right number of outputs for the new task. Similarly, the upper hidden layers of the original model are less likely to be as useful as the lower layers, since the high-level features that are most useful for the new task may differ significantly from the ones that were most useful for the original task. We have  to find the right number of layers to reuse: the more similar the tasks are, the more layers we can reuse (starting with the lower layers). For very similar tasks, we can try to keep all the hidden layers and just replace the output layer. Let's look at a simple example in order to show how transfer Learning can be implemented in Keras. Suppose someone has built a model on a simpler Fashion MNIST dataset (one with only eight classes, all the classes except for sandal and shirt). Let’s call this model A. We now want to tackle a different task: we have images of sandals and shirts, and we want to train a binary classifier (positive=shirt, negative=sandal). That task is quite similar to the first one, so we can try transfer learning. Let's split the fashion MNIST training set in two:
- X_train_A: all images of all items except for sandals and shirts (classes 5 and 6)
- X_train_B: a much smaller training set of just the first 200 images of sandals or shirts

In [27]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

In [28]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [29]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.InputLayer(shape=(28, 28)))
model_A.add(keras.layers.Flatten())
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="relu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [30]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(),
                metrics=["accuracy"])

In [31]:
history = model_A.fit(X_train_A, y_train_A, epochs=10, verbose=0,
                      validation_data=(X_valid_A, y_valid_A))

In [ ]:
print("Validation accuracy:", history.history["val_accuracy"][-1])

Now we create a new model (model B) based on that model A layers, let’s reuse all the layers except for the output layer. We can train model B for task B, but since the new output layer was initialized randomly, it will make large errors, so there will be large error gradients that may wreck the reused weights. To avoid this, one approach is to freeze the reused layers during the first few epochs, giving the new layer some time to learn reasonable weights. To do this, we have to set every layer’s trainable attribute to false, compile the model and train it for a few epochs; then unfreeze the reused layers (which requires compiling the model again) and continue training to fine-tune the reused layers for task B.

In [33]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B = keras.models.Sequential(model_A_clone.layers[:-1])
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [34]:
for layer in model_B.layers[:-1]:
    layer.trainable = False

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=4, verbose=0,
                      validation_data=(X_valid_B, y_valid_B))

In [35]:
for layer in model_B.layers[:-1]:
    layer.trainable = True

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=6, verbose=0,
                      validation_data=(X_valid_B, y_valid_B))

In [ ]:
print("Validation accuracy:", history.history["val_accuracy"][-1])

So, what’s the final verdict? We can compare with a network learned from scratch:

In [37]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.InputLayer(shape=(28, 28)))
model_B.add(keras.layers.Flatten())
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [38]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(),
                metrics=["accuracy"])

In [39]:
history = model_B.fit(X_train_B, y_train_B, epochs=10, verbose=0,
                      validation_data=(X_valid_B, y_valid_B))

In [ ]:
print("Validation accuracy:", history.history["val_accuracy"][-1])

In general, transfer learning **does not work very well with small dense networks**, presumably because small networks learn **few patterns**, and dense networks learn **very specific patterns**, which are unlikely to be useful in other tasks. Transfer learning works best with deep convolutional neural networks, which tend to learn feature detectors that are much more general (especially in the lower layers). 

## Faster Optimization

Training large DNNs can be painfully slow. A huge speed boost comes from using a faster optimizer than the regular Gradient Descent. The most popular algorithms are momentum optimization, Nesterov Accelerated Gradient, AdaGrad, RMSProp, Adam and Nadam optimizations.

### Momentum Optimization

Imagine a bowling ball rolling down a gentle slope on a smooth surface, it will start out slowly, but it will quickly pick up momentum until it eventually reaches terminal velocity. In contrast, regular Gradient Descent will simply take small, regular steps down the slope, so the algorithm will take much more time to reach the bottom. Recall that Gradient Descent updates the weights by subtracting the gradient of the cost function with regard to the weight multiplied by the learning rate:

$\displaystyle \theta_t = \theta_{t-1} - \eta \nabla J(\theta_{t-1})$

It does not care about what the earlier gradients were. If the local gradient is tiny, it goes very slowly. Momentum optimization takes into account also the value of previous gradients. At each iteration, it subtracts the local gradient from the momentum vector $m$ and it updates the weights by adding this momentum:

$\displaystyle m_t = \beta m_{t-1} - \eta \nabla J(\theta_{t-1})$

$\displaystyle \theta_t = \theta_{t-1} + m_t$

In other words, the gradient is **used for acceleration, not for speed**. To simulate some sort of friction mechanism and prevent the momentum from growing too large, the algorithm introduces a new hyperparameter $\beta$, called the **momentum**, which must be set between 0 (high friction) and 1 (no friction). A typical momentum value is 0.9.

![](images/momentum.png)

To visualize the difference between standard gradient descent and momentum gradient descent, we can plot the convergence of the loss function over iterations for both optimizers in the context of a simple cost function of two parameters. Its global minimum is inside a long, narrow valley:

$\displaystyle f(\theta_1,\theta_2) = a \theta_1^2 + b \theta_2^2$

In [41]:
def function(theta, a=20, b=1):
    return a * theta[0]**2 + b * theta[1]**2

We can plot the function in order to see the valley:

In [ ]:
theta_1 = np.linspace(-3, 3, 400)
theta_2 = np.linspace(-3, 3, 400)
X, Y = np.meshgrid(theta_1, theta_2)

F = function([X, Y])

plt.figure(figsize=(15, 6))
ax = plt.subplot(1, 2, 1, projection='3d') 
ax.plot_surface(X, Y, F, cmap='viridis')
ax.set_xlabel('theta_1')
ax.set_ylabel('theta_2')

plt.subplot(1, 2, 2) 
contour = plt.contourf(X, Y, F, levels=50, cmap='viridis')
plt.colorbar()
plt.xlabel('theta 1')
plt.ylabel('theta 2')

plt.show()

For tthis function, the gradient can be calculated analytically. The gradient vector is:

$\displaystyle \nabla f(\theta_1,\theta_2) = \begin{bmatrix} 2a \theta_1 \\ 2 b \theta_2 \end{bmatrix}$

In [43]:
def gradient(theta, a=20, b=1):
    df_d_theta_1 = 2 * a * theta[0]
    df_d_theta_2 = 2 * b * theta[1]
    
    gradient = np.array([df_d_theta_1, df_d_theta_2])
    return gradient

We initialize the starting point and the parameters: learning rate, momentum and the numer of iterations:

In [44]:
starting_point = [-3, -3]

learning_rate = 0.001
momentum = 0.95
n_iterations = 10000

We can consider the Rosenbrock function as a cost function and apply the standard gradient descent and the momentum gradient descent:

In [45]:
def standard_gradient_descent(cost_function, gradient_function, starting_point, learning_rate, n_iterations):
    theta = starting_point
    cost_history = []
    theta_history = []
    for iteration in range(n_iterations):
        gradient = gradient_function(theta)
        theta -= learning_rate * gradient
        cost = cost_function(theta)
        cost_history.append(cost)
        theta_history.append(theta.copy())
    return theta, cost_history, theta_history

In [46]:
def momentum_gradient_descent(cost_function, gradient_function, starting_point, learning_rate, n_iterations, momentum):
    theta = starting_point
    velocity = np.zeros_like(theta)
    cost_history = []
    theta_history = []
    for iteration in range(n_iterations):
        gradient = gradient_function(theta)
        velocity = momentum * velocity - learning_rate * gradient
        theta += velocity
        cost = cost_function(theta)
        cost_history.append(cost)
        theta_history.append(theta.copy())
    return theta, cost_history, theta_history

Run both gradient descent algorithms:

In [47]:
theta_sgd, cost_history_sgd, theta_history_sgd = standard_gradient_descent(function, gradient, starting_point, learning_rate, n_iterations)
theta_mgd, cost_history_mgd, theta_history_mgd = momentum_gradient_descent(function, gradient, starting_point, learning_rate, n_iterations, momentum)

Finally, we can print the final parameters learned and plot the cost history:

In [ ]:
print("Theta (Standard Gradient Descent):", theta_sgd)
print("Theta (Momentum Gradient Descent):", theta_mgd)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(cost_history_sgd, label="Standard Gradient Descent")
plt.plot(cost_history_mgd, label="Momentum Gradient Descent")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.xlim((0, 200))
plt.title("Cost History")
plt.legend()
plt.show()

Typically, the standard Gradient Descent shows a gradual decrease in cost over iterations, but it may take a longer time to reach the minimum cost. Momentum often converges faster, which helps in accelerating the learning. However, it can show oscillations, especially if the learning rate or the momentum factor are not tuned properly. The momentunm term can cause the update to overshoot the minimum if it accumulates too much momentum, leading to oscillations around the optimal value. In Keras we can use momentum hyperparameter with the SGD optimizer.
 

In [50]:
optimizer = keras.optimizers.SGD(momentum=0.9)

### Nesterov Accelerated Gradient

To reduce these oscillations, we can observe that in general the momentum vector will be pointing in the right direction (it "looks ahead" toward the optimum), so it will be slightly more accurate to use the gradient measured a bit farther in that direction rather than the gradient at the original position:

![](images/nesterov.png)

The standard momentum method first computes the gradient at the current location and then takes a jump in the direction of the updated accumulated gradient, in contrast Nesterov first makes a jump in the direction of the previous accumulated gradient and then measures the gradient where it ends up and makes a correction. The idea being that it is better to correct a mistake after we have made it:

$\displaystyle m_t = \beta m_{t-1} - \eta \nabla_{\theta} J(\theta + \beta m_{t-1})$

$\displaystyle \theta_t = \theta_{t-1} + m_t$

We can show the difference in the previous example:

In [51]:
def nesterov_gradient_descent(cost_function, gradient_function, starting_point, learning_rate, n_iterations, momentum):
    theta = starting_point
    velocity = np.zeros_like(theta)
    cost_history = []
    theta_history = []
    for iteration in range(n_iterations):
        lookahead = theta + momentum * velocity
        gradients = gradient_function(lookahead)
        velocity = momentum * velocity - learning_rate * gradients
        theta += velocity
        cost = cost_function(theta)
        cost_history.append(cost)
        theta_history.append(theta.copy())
    return theta, cost_history, theta_history

In [52]:
theta_nesterov, cost_history_nesterov, theta_history_nesterov = nesterov_gradient_descent(function, gradient, starting_point, learning_rate, n_iterations, momentum)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(cost_history_mgd, label="Momentum Gradient Descent")
plt.plot(cost_history_nesterov, label="Nesterov Gradient Descent")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.xlim((0, 100))
plt.title("Cost History")
plt.legend()
plt.show()

Typically, Nesterov Accelerated Gradient converges faster and more smoothly than both standard gradient descent and momentum, especially in the presence of complex optimization landscapes. In Keras, simply set nesterov=True when creating the SGD optimizer:

In [54]:
optimizer = keras.optimizers.SGD(momentum=0.9, nesterov=True)

### AdaGrad

In traditional gradient descent, a single global learning rate is used for all parameters and all iterations. This can be suboptimal because different parameters might require different learning rates. Imagine you are trying to optimize a function with two parameters, and suppose one has a steep gradient while the other has a flat gradient. In standard gradient descent, both are updated with the same learning rate, which might cause the first to oscillate and the second to update too slowly. [AdaGrad](https://www.jmlr.org/papers/volume12/duchi11a/duchi11a.p) **adapts the learning rate for each parameter based on the magnitude of its gradients**. Parameters with frequently gradients get a smaller learning rate, while parameters with infrequent gradients get a larger learning rate. It maintains a running sum of the squares of the gradients for each parameter. This accumulated value increases over time, especially for parameters with large gradients:

$\displaystyle g_{t,i} = g_{t-1,i} + \left(\frac{\partial J(\theta_{t-1,i})}{\partial \theta_{t-1,i}}\right)^2$

This accumulation causes the learning rate to shrink over time for parameters that frequently have large gradients. The effective learning rate for each parameter at any given iteration is: 

$\displaystyle \eta_{t,i} = \frac{\eta_{t-1,i}}{\sqrt{g_{t,i} + \epsilon}}$

In short, this algorithm decays the learning rate, but it does so faster for steep dimensions than for dimensions with gentler slopes. This is called an **adaptive learning rate**. It helps point the resulting updates more directly toward the global optimum.

![](images/adagrad.png)

AdaGrad frequently performs well for simple quadratic problems, but it often stops too early when training neural networks. The learning rate gets scaled down so much that the algorithm ends up stopping entirely before reaching the global optimum. We should not use it to train deep neural networks (it may be efficient for simpler tasks such as Linear Regression). Still, understanding AdaGrad is helpful to grasp the other adaptive learning rate optimizers.

In [55]:
def adagrad(cost_function, gradient_function, starting_point, learning_rate, n_iterations):
    epsilon=1e-8
    theta = starting_point
    g = np.zeros_like(theta, dtype=np.float64)
    cost_history = []
    theta_history = []
    for iteration in range(n_iterations):
        gradients = gradient_function(theta)
        g += gradients ** 2
        adjusted_learning_rate = learning_rate / (np.sqrt(g) + epsilon)
        theta -= adjusted_learning_rate * gradients
        cost = cost_function(theta)
        cost_history.append(cost)
        theta_history.append(theta.copy())
    return theta, cost_history, theta_history

In [56]:
theta_adagra, cost_history_adagrad, theta_history_adagrad = adagrad(function, gradient, starting_point, learning_rate, n_iterations)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(cost_history_sgd, label="Standard Gradient Descent")
plt.plot(cost_history_mgd, label="Momentum Gradient Descent")
plt.plot(cost_history_adagrad, label="Adagrad")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.xlim((0, 200))
plt.title("Cost History")
plt.legend()
plt.show()

The learning rate adaptation mechanism of AdaGrad makes it well-suited for dealing with sparse data and ensures that the learning rate decreases over time. However, this can also lead to very small learning rates, which can cause AdaGrad to converge more slowly than SGD in some cases. In Keras, we can use the AdaGrad optimizer by setting the optimizer parameter to 'adagrad':

In [58]:
optimizer = keras.optimizers.Adagrad()

### RMSProp

In order to fix the AdaGrad risk of slowing down too fast, we cna accumulatie only the gradients from the most recent iterations (as opposed to all the gradients since the beginning of training). The sum of squared gradients is multiplied by a decay rate and the current gradient is weighted by 1 minus the decay rate. The update step looks the same as in AdaGrad:

$\displaystyle g_{t,i} = \beta g_{t-1,i} + (1 - \beta) \left(\frac{\partial J(\theta_{t,i})}{\partial \theta_{t,i}}\right)^2$

The decay rate is typically set to 0.9, and it is a new hyperparameter, but this default value often works well, so we may not need to tune it at all. Except on very simple problems, this optimizer almost always performs much better than AdaGrad. 

In [59]:
def RMSProp(cost_function, gradient_function, starting_point, learning_rate, n_iterations, decay_rate):
    epsilon=1e-8
    theta = starting_point
    g = np.zeros_like(theta, dtype=np.float64)
    cost_history = []
    theta_history = []
    for iteration in range(n_iterations):
        gradients = gradient_function(theta)
        g = decay_rate * g + (1-decay_rate) * gradients ** 2
        adjusted_learning_rate = learning_rate / (np.sqrt(g) + epsilon)
        theta -= adjusted_learning_rate * gradients
        cost = cost_function(theta)
        cost_history.append(cost)
        theta_history.append(theta.copy())
    return theta, cost_history, theta_history

In [60]:
deacy_rate = 0.9

theta_rmsprop, cost_history_rmsprop, theta_history_rmsprop = RMSProp(function, gradient, starting_point, learning_rate, n_iterations, deacy_rate)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(cost_history_sgd, label="Standard Gradient Descent")
plt.plot(cost_history_mgd, label="Momentum Gradient Descent")
plt.plot(cost_history_adagrad, label="Adagrad")
plt.plot(cost_history_rmsprop, label="RMSProp")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.xlim((0, 200))
plt.title("Cost History")
plt.legend()
plt.show()

Although SGD with momentum is able to find the global minimum faster, this algorithm takes a much longer path that could be dangerous. This is because a longer path means more potential saddle points and local minima of the loss function that could lie along that path. RMSProp, on the other hand, goes straight to the global minimum of the loss function without taking a detour. We can see this by plotting the path of the different algothems:

In [62]:
import matplotlib as mpl
from matplotlib import animation

mpl.rc('animation', html='jshtml')
mpl.rcParams['animation.embed_limit'] = 2**128

fig = plt.figure(figsize=(12, 6))
ax = plt.axes(xlim=(-3, 3), ylim=(-3, 3))
plt.contourf(X, Y, F, levels=50, cmap='viridis')

def init():
    line_rmsprop.set_data([], [])
    line_sgd.set_data([], [])
    line_mgd.set_data([], [])
    return line_rmsprop, line_sgd, line_mgd
    
def animate(frame):
    line_rmsprop.set_data([point[0] for point in theta_history_rmsprop[:frame+1]], [point[1] for point in theta_history_rmsprop[:frame+1]])
    line_sgd.set_data([point[0] for point in theta_history_sgd[:frame+1]], [point[1] for point in theta_history_sgd[:frame+1]])
    line_mgd.set_data([point[0] for point in theta_history_mgd[:frame+1]], [point[1] for point in theta_history_mgd[:frame+1]])
    return line_rmsprop, line_sgd, line_mgd

line_rmsprop, = ax.plot([], [], linewidth=2, label="RMSProp")
line_sgd, = ax.plot([], [], linewidth=2, label="Standard Gradient Descent")
line_mgd, = ax.plot([], [], linewidth=2, label="Momentum Gradient Descent")
plt.legend(loc="upper left")

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=2500, interval=10, blit=True)
    
plt.close()

In [ ]:
anim

Keras has an RMSprop optimizer:

In [64]:
optimizer = keras.optimizers.RMSprop(rho=0.9)

### Adam

The idea behind momentum and RMSProp seemed pretty good, so can be combined in the [Adam](https://arxiv.org/abs/1412.6980) optimization approach: like momentum, it determines the velocity of the gradient and update the weight parameter in the direction of that velocity, like RMSProp it uses the sum of squared gradients to scale the current gradient to update the weights in each space dimension with the same ratio.

$\displaystyle m_t = \beta_1 m_{t-1} - (1-\beta_1) \nabla J(\theta_{t-1})$  (momentum)

$\displaystyle g_{t,i} = \beta_2 g_{t-1,i} + (1 - \beta_2) (\frac{\partial J(\theta_{t,i})}{\partial \theta_{t,i}})^2$ (RMSProp)

$\displaystyle \eta_{t,i} = \frac{\eta_{t-1,i}}{\sqrt{g_{t,i} + \epsilon}}$

$\displaystyle \theta_{t,i} = \theta_{t-1,i} + \eta_{t,i} m_t$ 

The first expression looks like momentum, the difference with is the factor multiplied by the current gradient. In the case of ADAM is called "first momentum". The second expression can be considered as RMSProp, where we keep the running sum of squared gradients. This term is called the "second momentum". The final update equation can be viewed as a combination of the two. The hyperparameters $\beta_1$ and $\beta_2$ are typically initialized to 0.9 and 0.999, respectively. 

In this way, Adam incorporats the nice features of the previous optimization algorithms. However, there is a problem at the beginning of training. At the first time step, the first and second momentum terms are set to zero. After the first update of the second momentum, this term is still very close to zero and when we update the weight parameters in the last expression, we divide by a small value and this leads to a large first update step, which is an artifact of the fact that we initialized the first and second momentum to zero. To address the problem of large update steps happening at the beginning of training, Adam includes a correction clause:

$\displaystyle \hat{m}_t = \frac{m_t}{1-\beta_1^t}$

$\displaystyle \hat{g}_t = \frac{g_t}{1-\beta_2^t}$

These correction cause the values of the first and second momentum to be higher at the beginning of the training. As a result, the first update step of the neural network weight parameters does not become too large.  

In [65]:
def adam(cost_function, gradient_function, starting_point, learning_rate, n_iterations, beta1, beta2):
    theta = starting_point

    epsilon=1e-8
    
    m = np.zeros_like(theta) 
    g = np.zeros_like(theta)
    
    cost_history = []
    theta_history = []

    for t in range(1, n_iterations + 1):
        gradient = gradient_function(theta)

        m = beta1 * m - (1 - beta1) * gradient
        g = beta2 * g + (1 - beta2) * (gradient ** 2)
         
        theta += learning_rate * m / (np.sqrt(g) + epsilon)
        
        cost = cost_function(theta)
        cost_history.append(cost)
        theta_history.append(theta.copy())

    return theta, cost_history, theta_history


In [66]:
beta1=0.9
beta2=0.999

theta_adam, cost_history_adam, theta_history_adam = adam(function, gradient, starting_point, learning_rate, n_iterations, beta1, beta2)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(cost_history_sgd, label="Standard Gradient Descent")
plt.plot(cost_history_mgd, label="Momentum Gradient Descent")
plt.plot(cost_history_adagrad, label="Adagrad")
plt.plot(cost_history_rmsprop, label="RMSProp")
plt.plot(cost_history_adam, label="Adam")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.xlim((0, 200))
plt.title("Cost History")
plt.legend()
plt.show()

We can mention a variants of Adam, called [Nadam](http://cs229.stanford.edu/proj2015/054_report.pdf)) which include the the Nesterov trick in Adam. Keras has an Adam optimizer:

In [68]:
optimizer = keras.optimizers.Adam(beta_1=0.9, beta_2=0.999)

All the optimization techniques discussed so far only rely on the first-order partial derivatives (**Jacobians**). The optimization literature also contains powerful algorithms based on the second-order partial derivatives (the **Hessians**, which are the partial derivatives of the Jacobians). Unfortunately, these algorithms are very hard to apply to deep neural networks because there are $n^2$ Hessians per output (where $n$ is the number of parameters), as opposed to just $n$ Jacobians per output. Since DNNs typically have tens of thousands of parameters, the second-order optimization algorithms often don’t even fit in memory, and even when they do, computing the Hessians is just too slow.

All these adaptive optimization methods  are often great, converging fast to a good solution. However, the paper [The Marginal Value of Adaptive Gradient Methods in Machine Learning](https://arxiv.org/abs/1705.08292) showed that they can lead to **solutions that generalize poorly** on some datasets. So when we are disappointed by our model’s performance, we can try using plain Nesterov Accelerated Gradient instead: our  dataset may just be allergic to adaptive gradients. Also check out the latest research, because it’s moving fast.

## Learning Rate Scheduling

Finding a good learning rate is very important: 

- if we set it much too high, training may diverge
- if we set it too low, training will eventually converge to the optimum, but it will take a very long time
- if we set it slightly too high, it will make progress very quickly at first, but it will end up dancing around the optimum, never really settling down

We can find a good learning rate by training the model for a few hundred iterations, exponentially increasing the learning rate from a very small value to a very large value, and then looking at the learning curve and picking a learning rate slightly lower than the one at which the learning curve starts shooting back up. We can then reinitialize our model and train it with that learning rate.

![](images/learning-rates.png)

But we can do better than a constant learning rate: if we start with a large learning rate and then we reduce it once training stops making fast progress, we can reach a good solution faster than with a constant learning rate. 

![](images/learning-schedule.png)

There are many different strategies, called **learning schedule** to reduce the learning rate during training.

### Power scheduling

This approach ensures that the learning rate decreases smoothly according to a power law that sets the learning rate to a function of the iteration number $t$: 

$\displaystyle \eta(t) = \frac{\eta_0}{(1+s \cdot t)^c}$

where $s$ is the decay rate and controls how quickly the learning rate decreases and $c"$ is the power hyperparameter, that controls the shape of the decay curve.

In [69]:
def power_scheduler(learning_rate_0, decay, power, step):
    learning_rate = learning_rate_0 / (1 + decay * step)**power
    return learning_rate

We can try some different values of the hyperparameters and plot the learning rate over the iterations:

In [70]:
starting_learning_rate = 0.1
power = 1
max_steps = 1000

steps = np.arange(max_steps)

learning_rates_1 = power_scheduler(starting_learning_rate, 0.01, 1, steps)
learning_rates_2 = power_scheduler(starting_learning_rate, 0.05, 1, steps)
learning_rates_3 = power_scheduler(starting_learning_rate, 0.10, 1, steps)
learning_rates_4 = power_scheduler(starting_learning_rate, 0.01, 2, steps)
learning_rates_5 = power_scheduler(starting_learning_rate, 0.05, 2, steps)
learning_rates_6 = power_scheduler(starting_learning_rate, 0.10, 2, steps)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(steps, learning_rates_1, label='decay=0.01, power=1')
plt.plot(steps, learning_rates_2, label='decay=0.05, power=1')
plt.plot(steps, learning_rates_3, label='decay=0.10, power=1')
plt.plot(steps, learning_rates_4, label='decay=0.01, power=2')
plt.plot(steps, learning_rates_5, label='decay=0.05, power=2')
plt.plot(steps, learning_rates_6, label='decay=0.10, power=2')

plt.xlabel('Steps')
plt.ylabel('Learning Rate')
plt.title('Power Scheduler')
plt.legend()
plt.show()

Here, the iteration number can be a step (mini-batch) or an epoch, the decision depends on the specific needs of the training process and the behavior of the model. in general, changing the learning rate after each step allows for more granular control, potentially leading to smoother and more consistent training progress, however frequent changes might also introduce instability, particularly if the batch size is small or if the training data is highly variable. Changing the learning rate after each epoch is simpler to implement and understand, it tends to be more stable (since the learning rate remains constant within an epoch), which might help in achieving more consistent updates during training. The downside is that it might adapt more slowly to the changing dynamics of the training process.

### Exponential scheduling

We can decrease the learning rate exponentially over time:

$\displaystyle \eta(t) = \eta_0 \cdot e^{-s \cdot t}$ 

While power scheduling decays at a rate determined by a polynomial function, initially decreasing quickly and then slowing down, exponential scheduling decays at a constant rate, decreasing the learning rate by a constant factor at each step. This can be useful when we want to quickly reduce the learning rate at the beginning of training, but we don’t want to decrease it too slowly later on. It is simpler (with fewer parameters), but less flexible in controlling the shape of the decay.

In [72]:
def exponential_scheduler(learning_rate_0, decay, steps):
    learning_rate =  learning_rate_0 * np.exp(-decay * steps)
    return learning_rate

In [73]:
learning_rates_1 = exponential_scheduler(starting_learning_rate, 0.01, steps)
learning_rates_2 = exponential_scheduler(starting_learning_rate, 0.05, steps)
learning_rates_3 = exponential_scheduler(starting_learning_rate, 0.1, steps)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(steps, learning_rates_1, label='decay=0.01')
plt.plot(steps, learning_rates_2, label='decay=0.05')
plt.plot(steps, learning_rates_3, label='decay=0.10')

plt.xlabel('Steps')
plt.ylabel('Learning Rate')
plt.title('Exponential Scheduler')
plt.legend()
plt.show()

### Piecewise constant scheduling

Unlike exponential and power scheduling, this scheduling keeps the learning rate constant for specified intervals and then changes it to a new constant value at predefined steps, allowing for manual control over the learning rate schedule. This method is highly flexible as it allows setting specific learning rates at different stages of training, often used to decrease the learning rate after the model has plateaued. Although this solution can work very well, it requires fiddling around to figure out the right sequence of learning rates and how long to use each of them.

$\displaystyle \eta(t) = \eta_0 \cdot 10^{-\text{floor}(t/t_1)}$

In [75]:
def piecewise_constant_scheduler(learning_rate_0, boundaries, values, steps):
    learning_rates = np.full_like(steps, learning_rate_0, dtype=np.float64)
    for boundary, value in zip(boundaries, values):
        learning_rates[steps >= boundary] = value
    return learning_rates    

In [76]:
learning_rate_0 = 0.1
boundaries = [0, 300, 600, 900]
values = [0.1, 0.01, 0.001, 0.0001]

learning_rates = piecewise_constant_scheduler(learning_rate_0, boundaries, values, steps)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(steps, learning_rates)
plt.xlabel('Steps')
plt.ylabel('Learning Rate')
plt.title('Piecewise Constant Scheduler')
plt.show()

### Performance scheduling

It adjusts the learning rate based on the performance of the model (such as the validation loss or accuracy), rather than on a predefined schedule. Typically this method involves monitoring the performance on a validation set and reducing the learning rate when performance stagnates or deteriorates. Often includes a patience parameter that specifies how many steps to wait before reducing the learning rate after a performance plateau is detected. This method can be more responsive to the training dynamics, potentially leading to better convergence and performance, and reduces the need for manually tuning learning rate schedules. However, it is more complex to implement and requires monitoring validation performance.

In [78]:
def performance_scheduler(initial_learning_rate, val_losses, patience=10, factor=0.1, min_learning_rate=1e-6):
    learning_rates = []
    best_loss = np.inf
    wait = 0
    current_learning_rate = initial_learning_rate
    
    for val_loss in val_losses:
        if val_loss < best_loss:
            best_loss = val_loss
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                new_learning_rate = max(current_learning_rate * factor, min_learning_rate)
                if current_learning_rate > new_learning_rate:
                    current_learning_rate = new_learning_rate
                wait = 0
        learning_rates.append(current_learning_rate)
    
    return learning_rates


In order to show the effect of this scheduler, we simulate a validation cost that decreases over time and we apply the scheduler:

In [79]:
learning_rate_0 = 0.5

# Simulated validation losses
val_losses = np.linspace(0.5, 0.3, max_steps) + np.random.uniform(0.1, 0.2, max_steps) 

learning_rates = performance_scheduler(learning_rate_0, val_losses, patience=30, factor=0.5, min_learning_rate=1e-4)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(steps, learning_rates, label="learning_rate")
plt.plot(steps, val_losses, label="simulated loss")
plt.xlabel('Steps')
plt.ylabel('Learning Rate')
plt.title('Performance Scheduler')
plt.legend()
plt.show()

Determining the "best" learning rate scheduler depends on various factors, including the specific characteristics of the model, the dataset, and the training objectives. Each learning rate scheduler has its own advantages and is suited to different scenarios. As an example, the paper [An Empirical Study of Learning Rates in Deep Neural Networks for Speech Recognition](https://ieeexplore.ieee.org/document/6638963) compared the performance of some of the most popular learning schedules when using momentum optimization to train deep neural networks for speech recognition. The authors concluded that, in this setting, both performance scheduling and exponential scheduling performed well. They favored exponential scheduling because it was easy to tune and it converged slightly faster to the optimal solution.

## Regularization

DNNs typically have tens of thousands of parameters (even millions an in the case of large language model trillions). This gives them an incredible amount of freedom and means they can fit a huge variety of complex datasets. But this great flexibility also makes the network **prone to overfitting** the training set. For this reasion, regularization techniquesa are used to constrain a model, reducing its freedom and preventing it from overfitting the training set. 

### Early stopping

Early stopping is a form of regularization used to avoid overfitting when training a learner with an iterative method, such as gradient descent. During training, the model performance is monitored on both the training set and a separate validation set. The validation set is used to simulate how the model performs on new, unseen data. An "early stopping" message is triggered if the performance on the validation set stops improving. A **patience** parameter is often used to define how many epochs to wait after the last improvement before stopping the training. In this way, we ensure that the training is stopped before the model starts to overfit the training data, ensuring better generalization to new data and saving time and resource during training. 

![](images/early-stopping.png)

Most deep learning frameworks provide built-in mechanisms for early stopping. For instance, in Keras:


In [81]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [82]:
model = keras.models.Sequential([
    keras.layers.InputLayer(shape=(28, 28)),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [83]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(),
              metrics=["accuracy"])

In [84]:
history = model.fit(X_train, y_train, epochs=250, verbose=0,
                    validation_data=(X_valid, y_valid),
                    callbacks=[callback])

In [ ]:
print("Early stopping epoch: ", len(history.history['loss']))

### L1 and L2 regularization

Just like for simple linear models, we can use Ridge (L2), Lasso (L1) or Elastinc Net (combination of L1 and L2) regularization to constrain connection weights. In these methods a regularization term is added to the cost function to forces the learning algorithm to not only fit the data but also keep the model weights as small as possible:

$\displaystyle L_{L2}(\theta) = L(\theta) + \alpha \sum\limits_{i=1}^{n}\theta^2_i$ (L2 penalty, **Ridge Regularization**)

$\displaystyle L_{L1}(\theta) = L(\theta) + \alpha \sum\limits_{i=1}^{n}\left|\theta_i\right|$ (L1 penalty **Lasso Regularization**)
 
$\displaystyle L_{L1,L2}(\theta) = L(\theta) + \alpha_1 \sum\limits_{i=1}^{n}\left|\theta_i\right| + \alpha_2 \sum\limits_{i=1}^{n}\theta^2_i$ (**Elastic Net**)

L2 regularization tends to shrink the weights but generally does not drive them to exactly zero, L1 regularization tends to drive some weights to exactly zero, effectively performing feature selection. Elastic Net is a middle ground between L1 and L2 regularization, it benefits from the feature selection properties of L1 regularization and the stability of L2 regularization. In Keras we can use L1 and L2 regularization by setting the kernel_regularizer parameter when creating a layer:

In [86]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

### Dropout

The core idea of [Dropout](https://dl.acm.org/doi/10.5555/2627435.2670313) is to randomly drop units (along with their connections) from the neural network during training, in order to prevent the network from becoming too dependent on any particular neuron, thus promoting redundancy and robustness in the network's learned features. The fraction of the input units to drop is an hyperparameter that can be tuned. At inference time, no units are dropped out, and instead the layer’s output values are scaled down by a factor equal to the dropout rate, to balance for the fact that more units are active than during training.

![](images/drop-out.png)

Neurons trained with dropout cannot co-adapt with their neighbors, they have to be as useful as possible on their own. They also cannot rely excessively on just a few input neurons, they must pay attention to each of their inputs. So neurons end up being less sensitive to slight changes in the inputs. In the end, we get a more robust network that generalizes better. Another way to understand the power of dropout is to realize that a unique neural network is generated at each training step. Since each neuron can be either present or absent, there are a total of $2^N$ possible networks (where N is the total number of droppable neurons). These neural networks are obviously not independent, because they share many of their weights, but they are nevertheless all different. The resulting neural network can be seen as an averaging ensemble of all these smaller networks. To implement dropout using Keras, we can use the specific layer:

In [87]:
model = keras.models.Sequential([
    keras.layers.InputLayer(shape=(28, 28)),    
    keras.layers.Flatten(),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

If we observe that the model is overfitting, we can increase the dropout rate. Conversely, we should try decreasing the dropout rate if the model underfits the training set. It can also help to increase the dropout rate for large layers, and reduce it for small ones. Moreover, many state-of-the-art architectures only use dropout after the last hidden layer, so we may want to try this if full dropout is too strong. Dropout does tend to significantly slow down convergence, but it usually
results in a much better model when tuned properly. So, it is generally well worth the extra time and effort.

We can also leverage Dropout during inference, in order to reason about the model uncertainty and to improve its performance. [**Monte Carlo Dropout**](https://arxiv.org/abs/1506.02142) applies dropout during inference and makes multiple forward passes for a single input, each time with different neurons dropped out according to the dropout rate. This results in multiple predictions for the same input.  This can be interpreted as an approximation of a gaussina probabilistic model. We can treat the many different networks (with different neurons dropped out) as Monte Carlo samples from the space of all available models. This provides mathematical grounds to reason about the model’s uncertainty and, as it turns out, often improves its performance. To perform inference with dropout active in Keras, we need to manually set the dropout layers to be active during inference:

In [91]:
input = X_test[:1]

dropout_predictions = np.array([model(input, training=True) for sample in range(100)])

The mean of these predictions can be taken as the final prediction and the variance or standard deviation can be used as a measure of the model's uncertainty. This can be useful in many applications, such as in medical diagnosis, where the model can provide a prediction along with a measure of confidence.

In [ ]:
dropout_prediction = dropout_predictions.mean(axis=0)
uncertainty = dropout_predictions.std(axis=0)

print("Dropout Prediction: ", np.round(dropout_prediction,2))
print("Uncertainty (std deviation): ", np.round(uncertainty,2))

We can compare with the prediction of a model without dropout:

In [ ]:
prediction = model.predict(input, verbose=0)
print("Prediction: ", np.round(prediction,2))

The model still thinks this example belongs to the same class, but using MC Dropout, it does so provinding also a measure of uncertainty. Apparently there’s quite a lot of variance in the probability estimates: if we were building a risk-sensitive system (e.g., a medical or financial system), we should probably treat such an uncertain prediction with extreme caution. The number of samples to use (100 in this example) is a hyperparameter we can tweak. The higher it is, the more accurate the predictions and their uncertainty estimates will be. However, inference time will increase. Moreover, above a certain number of samples, we will notice little improvement. So our job is to find the right trade-off between latency and accuracy, depending on our application.

### MaxNorm Regularization

Instead of adding a regularization term to the loss function, we can directly constrains the weights of each neuron: 

$\displaystyle \left\|w\right\|_2 < r$

where $r$ is an hyperparameter. It is typically implemented by rescaling weights after eaxh step Reducing the hyperparameter increases the amount of regularization and helps reduce overfitting Max-norm regularization can also alleviate the unstable gradients problem. In Keras, we can set the kernel_constraint argument of each hidden layer to a max_norm() constraint with the appropriate max value:

In [95]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                   kernel_constraint=keras.constraints.max_norm(1.));

### Data Augmentation

Data augmentation artificially increases the size of the training set by generating many realistic variants of each training instance. This
reduces overfitting, making this a regularization technique. The generated instances should be as realistic as possible: ideally, given an
image from the augmented training set, a human should not be able to tell whether it was augmented or not. Simply adding white noise will
not help; the modifications should be learnable (white noise is not).
For example, you can slightly shift, rotate, and resize every picture in
the training set by various amounts and add the resulting pictures to the
training set (see Figure 14-13). To do this, you can use Keras’s data
augmentation layers, introduced in Chapter 13 (e.g., RandomCrop,
RandomRotation, etc.). This forces the model to be more tolerant to
variations in the position, orientation, and size of the objects in the
pictures. To produce a model that’s more tolerant of different lighting
conditions, you can similarly generate many images with various
contrasts. In general, you can also flip the pictures horizontally (except
for text, and other asymmetrical objects). By combining these
transformations, you can greatly increase your training set size.


Data augmentation is also useful when you have an unbalanced dataset:
you can use it to generate more samples of the less frequent classes.
This is called the synthetic minority oversampling technique, or
SMOTE for short.

## Exercise

**Train a deep neural network (20 hidden layers of 100 neurons each: that’s too many, but it’s the point of this exercise; use He initialization and the ELU activation function) on the CIFAR10 image dataset. The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes. You can load it with keras.datasets.cifar10.load_ data(). (1) You have to Use the Nadam optimizator and the early stopping regularization strategy. (2) Then try adding Batch Normalization and compare the learning curves. (3) Then try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self normalizes (standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers). (4) Finally, try regularizing the model with alpha dropout**

In [96]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

Build a DNN with 20 hidden layers of 100 neurons each using He initialization and the ELU activation function:

In [106]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(32, 32,3)))
model.add(keras.layers.Flatten())
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

We use Nadam optimization and early stopping to train the network on the CIFAR10 datase:

In [107]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
callbacks = [early_stopping_cb]

result = model.fit(X_train, y_train, epochs=100, verbose=0,
                   validation_data=(X_valid, y_valid),
                   callbacks=callbacks);

In [ ]:
print("Validation accuracy:", result.history["val_accuracy"][-1])
print("Early stopping epoch: ", len(result.history['loss']))

Now we add Batch Normalization:

In [115]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(32, 32,3)))
model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [116]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [117]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
callbacks = [early_stopping_cb]

result = model.fit(X_train, y_train, epochs=100, verbose=0,
                   validation_data=(X_valid, y_valid),
                   callbacks=callbacks)

In [ ]:
print("Validation accuracy:", result.history["val_accuracy"][-1])
print("Early stopping epoch: ", len(result.history['loss']))

The model is converging faster than before in terns of number of epochs, however it is slower in each epoch due to the added layers. The BN layers stabilized training and allowed us to use a much larger learning rate, so convergence was faster. The final model is also better in terms of accuracy. It's still not a very good model, but at least it's much better than before.

Now we replace Batch Normalization with SELU:

In [119]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [120]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(32, 32,3)))
model.add(keras.layers.Flatten())
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [121]:
optimizer = keras.optimizers.Nadam(learning_rate=7e-4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [122]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
callbacks = [early_stopping_cb]

result = model.fit(X_train_scaled, y_train, epochs=100, verbose=0,
                   validation_data=(X_valid_scaled, y_valid),
                   callbacks=callbacks)

In [ ]:
print("Validation accuracy:", result.history["val_accuracy"][-1])
print("Early stopping epoch: ", len(result.history['loss']))

We get similar accuracy as the original model and it is much faster than both the original model and the BN model, plus each epoch took a short time just like the original model. So it's by far the fastest model to train (both in terms of epochs and wall time).

Now we regularize the model with dropout:

In [124]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(32, 32,3)))
model.add(keras.layers.Flatten())
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

In [125]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [126]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
callbacks = [early_stopping_cb]

result = model.fit(X_train_scaled, y_train, epochs=100, verbose=0,
                   validation_data=(X_valid_scaled, y_valid),
                   callbacks=callbacks)

In [ ]:
print("Validation accuracy:", result.history["val_accuracy"][-1])
print("Early stopping epoch: ", len(result.history['loss']))

The model is slightly worse than without dropout. With an extensive hyperparameter search, it might be possible to do better, but probably not much better in this case.